# Сложные объединения
---

Вы уже умеете присоединять строки друг к другу путём добавления столбцов «сбоку» — с помощью различных видов `JOIN`.

А что если нам необходимо присоединить несколько результатов «снизу», так, чтобы получить общий результат в выводе?

Ответу на этот вопрос и посвящён текущий модуль.

## Знакомство с данными
---

В данном блоке мы будем работать с данными о компании, организующей перевозки грузов.

Интересующие нас данные хранятся в таблицах `city`, `customer`, `driver`, `shipment`, `truck`. Давайте внимательно их изучим.

Ниже представлена *ER*-диаграмма (от англ. *entity-relation*, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

![](data/dst3-u2-md4_1_1.jpg)

Таблица `city` — это справочник городов. Структура справочника представлена ниже.

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                     ОПИСАНИЕ                    |
|:-------------:|:----------:|:-----------------------------------------------:|
| city_id       | integer    | уникальный идентификатор города, первичный ключ |
| city_name     | text       | название города                                 |
| state         | text       | штат, к которому относится город                |
| population    | integer    | население города                                |
| area          | numeric    | площадь города                                  |

Таблица `customer` — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

|  НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                      ОПИСАНИЕ                     |
|:--------------:|:----------:|:-------------------------------------------------:|
| cust_id        | integer    | уникальный идентификатор клиента, первичный ключ  |
| cust_name      | text       | название клиента                                  |
| annual_revenue | numeric    | ежегодная выручка                                 |
| cust_type      | text       | тип пользователя                                  |
| address        | text       | адрес                                             |
| zip            | integer    | почтовый индекс                                   |
| phone          | text       | телефон                                           |
| city_id        | integer    | идентификатор города, внешний ключ к таблице city |

Следующая таблица — `driver` — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                          ОПИСАНИЕ                          |
|:-------------:|:----------:|:----------------------------------------------------------:|
| driver_id     | integer    | уникальный идентификатор водителя, первичный ключ          |
| first_name    | text       | имя водителя                                               |
| last_name     | text       | фамилия водителя                                           |
| address       | text       | адрес водителя                                             |
| zip_code      | integer    | почтовый индекс водителя                                   |
| phone         | text       | телефон водителя                                           |
| city_id       | integer    | идентификатор города водителя, внешний ключ к таблице city |

В таблице `truck` хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                      ОПИСАНИЕ                      |
|:-------------:|:----------:|:--------------------------------------------------:|
| truck_id      | integer    | Уникальный идентификатор грузовика, первичный ключ |
| make          | text       | Производитель грузовика                            |
| model_year    | integer    | Дата выпуска грузовика                             |

Последняя таблица в датасете, `shipment`, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                                        ОПИСАНИЕ                                       |
|:-------------:|:----------:|:-------------------------------------------------------------------------------------:|
| ship_id       | integer    | уникальный идентификатор доставки, первичный ключ                                     |
| cust_id       | integer    | идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer  |
| weight        | numeric    | вес посылки                                                                           |
| truck_id      | integer    | идентификатор грузовика, на котором отправлена доставка, внешний ключ к таблице truck |
| driver_id     | integer    | идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver   |
| city_id       | integer    | идентификатор города в который совершена доставка, внешний ключ к таблице city        |
| ship_date     | date       | дата доставки                                                                         |

## Задание 1.1

> Укажите название города с максимальным весом единичной доставки.


```sql
SELECT
    city.city_name
FROM
    sql.shipment AS shipment
    JOIN sql.city AS city ON shipment.city_id = city.city_id
GROUP BY
    city.city_id,
    shipment.weight
ORDER BY
    shipment.weight DESC
LIMIT
    1
```

**Ответ:** Green Bay



> Сколько различных производителей грузовиков перечислено в таблице `truck`?


```sql
SELECT
    COUNT(DISTINCT truck.make)
FROM
    sql.truck AS truck
```

**Ответ:** 3

> Как зовут водителя (`first_name`), который совершил наибольшее количество доставок одному клиенту?


```sql
SELECT
    CONCAT(driver.first_name, ' ', driver.last_name)
FROM
    sql.shipment AS shipment
    JOIN sql.driver AS driver ON shipment.driver_id = driver.driver_id
GROUP BY
    shipment.cust_id,
    shipment.driver_id,
    driver.first_name,
    driver.last_name
ORDER BY
    COUNT(shipment.driver_id) DESC
LIMIT
    1
```

**Ответ:** Holger (Nohr)

> Укажите даты первой и последней по времени доставок в таблице `shipment`.
>
> Ответ введите в формате ДД.ММ.ГГГГ.


```sql
SELECT
    shipment.ship_date
FROM
    sql.shipment AS shipment
ORDER BY
    shipment.ship_date ASC
LIMIT
    1
```

Ответ: 08.01.2016 27.12.2017

> Укажите имя клиента, получившего наибольшее количество доставок за 2017 год.


```sql
SELECT
    customer.cust_name
FROM
    sql.shipment AS shipment
    JOIN sql.customer AS customer ON shipment.cust_id = customer.cust_id
GROUP BY
    shipment.cust_id,
    customer.cust_name
ORDER BY
    COUNT(shipment.cust_id) DESC
LIMIT
    1
```

**Ответ:** Autoware In